# Практическая работа

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

## Задание 1

Соберите всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

In [221]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

In [137]:
def main_info(name, dataset, id_column, show_nans=True):
    print(f'\n{name}: {dataset.shape}')
    if show_nans:
        print(f'Number of NAs: \n{dataset.isna().sum()}') 
    print(f'\nNumber of duplicates: {dataset.duplicated().sum()}')
    print(f'Number of duplicates (only ID): {dataset[id_column].duplicated().sum()}')

In [ ]:
'''
D_clients:
    - AGE — возраст клиента;
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;

D_agreement:
    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);

D_salary:    
    - PERSONAL_INCOME — личный доход клиента (в рублях);

D_loan & D_close_loan
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.
'''

### Clients, Salary and Agreement(Target)

In [138]:
clients = pd.read_csv('datasets/D_clients.csv')
salary = pd.read_csv('datasets/D_salary.csv')
agreement = pd.read_csv('datasets/D_target.csv')

loans = pd.read_csv('datasets/D_loan.csv')
close_loans = pd.read_csv('datasets/D_close_loan.csv')

In [139]:
main_info('Clients', clients, 'ID')
main_info('Salary', salary, 'ID_CLIENT')
main_info('Agreement', agreement, 'ID_CLIENT')


Clients: (16000, 14)
Number of NAs: 
ID                         0
AGE                        0
GENDER                     0
EDUCATION                  0
MARITAL_STATUS             0
CHILD_TOTAL                0
DEPENDANTS                 0
SOCSTATUS_WORK_FL          0
SOCSTATUS_PENS_FL          0
REG_ADDRESS_PROVINCE       0
FACT_ADDRESS_PROVINCE      0
POSTAL_ADDRESS_PROVINCE    0
FL_PRESENCE_FL             0
OWN_AUTO                   0
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 0

Salary: (15523, 3)
Number of NAs: 
FAMILY_INCOME      0
PERSONAL_INCOME    0
ID_CLIENT          0
dtype: int64

Number of duplicates: 300
Number of duplicates (only ID): 300

Agreement: (15223, 3)
Number of NAs: 
AGREEMENT_RK    0
ID_CLIENT       0
TARGET          0
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 0


In [142]:
salary.drop_duplicates(inplace=True)
main_info('Salary', salary, 'ID_CLIENT', show_nans=False)


Salary: (15223, 3)

Number of duplicates: 0
Number of duplicates (only ID): 0


In [143]:
clients_and_salary = clients.merge(salary,
                                   how='inner',
                                   left_on='ID',
                                   right_on='ID_CLIENT',
                                   suffixes=(False, False)).drop(['ID_CLIENT'],
                                                                 axis=1)

main_info('Clients and Salary', clients_and_salary, 'ID', show_nans=False)


Clients and Salary: (15223, 16)

Number of duplicates: 0
Number of duplicates (only ID): 0


In [191]:
clients_and_salary_and_agreement = clients_and_salary.merge(
    agreement,
    how='inner',
    left_on='ID',
    right_on='ID_CLIENT',
    suffixes=(False, False)).drop(['ID_CLIENT'], axis=1)

main_info('Clients and Salary and Agreement', clients_and_salary_and_agreement, 'ID', show_nans=False)


Clients and Salary and Agreement: (15223, 18)

Number of duplicates: 0
Number of duplicates (only ID): 0


### Loans

> **Комменатрий:** Есть наблюдения с одним и тем же значением ID_CLIENT, но это логично, так как у одного клиента может быть несколько кредитов

In [150]:
main_info('Loans', loans, 'ID_CLIENT')
main_info('Close loans', close_loans, 'ID_LOAN')


Loans: (21126, 2)
Number of NAs: 
ID_LOAN      0
ID_CLIENT    0
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 5903

Есть наблюдения с одним и тем же значением ID_CLIENT,
но это логично, так как у одного клиента может быть 
несколько кредитов


Close loans: (21126, 2)
Number of NAs: 
ID_LOAN      0
CLOSED_FL    0
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 0


In [156]:
loans_full = loans.merge(
    close_loans,
    how='inner',
    left_on='ID_LOAN',
    right_on='ID_LOAN',
    suffixes=(False, False))

main_info('Loans full info', loans_full, 'ID_CLIENT', show_nans=False)


Loans full info: (21126, 3)

Number of duplicates: 0
Number of duplicates (only ID): 5903


In [187]:
total_loans_num = loans_full[['ID_CLIENT']]\
                    .groupby('ID_CLIENT',as_index=False)\
                    .value_counts()\
                    .rename(columns={'count': 'LOAN_NUM_TOTAL'})

main_info('Total number of loans', total_loans_num, 'ID_CLIENT', show_nans=False)


Total number of loans: (15223, 2)

Number of duplicates: 0
Number of duplicates (only ID): 0


In [189]:
closed_loans_num = loans_full[['ID_CLIENT', 'CLOSED_FL']]\
                    .groupby(['ID_CLIENT', 'CLOSED_FL'],as_index=False)\
                    .value_counts()\
                    .query('CLOSED_FL == 1')\
                    .drop(['CLOSED_FL'], axis=1)\
                    .rename(columns={'count': 'LOAN_NUM_CLOSED'})

main_info('Number of closed loans', closed_loans_num, 'ID_CLIENT', show_nans=False)


Number of closed loans: (7432, 2)

Number of duplicates: 0
Number of duplicates (only ID): 0


### Final dataset

In [ ]:
clients_and_salary_and_agreement
total_loans_num
closed_loans_num

In [197]:
df = clients_and_salary_and_agreement.merge(
    total_loans_num,
    how='inner',
    left_on='ID',
    right_on='ID_CLIENT',
    suffixes=(False, False)).drop(['ID_CLIENT'], axis=1)

main_info('Clients and Salary and Agreement and Number of loans', df, 'ID', show_nans=True)


Clients and Salary and Agreement and Number of loans: (15223, 19)
Number of NAs: 
ID                         0
AGE                        0
GENDER                     0
EDUCATION                  0
MARITAL_STATUS             0
CHILD_TOTAL                0
DEPENDANTS                 0
SOCSTATUS_WORK_FL          0
SOCSTATUS_PENS_FL          0
REG_ADDRESS_PROVINCE       0
FACT_ADDRESS_PROVINCE      0
POSTAL_ADDRESS_PROVINCE    0
FL_PRESENCE_FL             0
OWN_AUTO                   0
FAMILY_INCOME              0
PERSONAL_INCOME            0
AGREEMENT_RK               0
TARGET                     0
LOAN_NUM_TOTAL             0
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 0


In [198]:
df = df.merge(
    closed_loans_num,
    how='left',
    left_on='ID',
    right_on='ID_CLIENT',
    suffixes=(False, False)).drop(['ID_CLIENT'], axis=1)

main_info('Clients and Salary and Agreement and Full info about loans', df, 'ID', show_nans=True)


Clients and Salary and Agreement and Full info about loans: (15223, 20)
Number of NAs: 
ID                            0
AGE                           0
GENDER                        0
EDUCATION                     0
MARITAL_STATUS                0
CHILD_TOTAL                   0
DEPENDANTS                    0
SOCSTATUS_WORK_FL             0
SOCSTATUS_PENS_FL             0
REG_ADDRESS_PROVINCE          0
FACT_ADDRESS_PROVINCE         0
POSTAL_ADDRESS_PROVINCE       0
FL_PRESENCE_FL                0
OWN_AUTO                      0
FAMILY_INCOME                 0
PERSONAL_INCOME               0
AGREEMENT_RK                  0
TARGET                        0
LOAN_NUM_TOTAL                0
LOAN_NUM_CLOSED            7791
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 0


In [203]:
df['LOAN_NUM_CLOSED'] = df['LOAN_NUM_CLOSED'].fillna(0).astype(int)

main_info('Clients and Salary and Agreement and Full info about loans', df, 'ID', show_nans=True)


Clients and Salary and Agreement and Full info about loans: (15223, 20)
Number of NAs: 
ID                         0
AGE                        0
GENDER                     0
EDUCATION                  0
MARITAL_STATUS             0
CHILD_TOTAL                0
DEPENDANTS                 0
SOCSTATUS_WORK_FL          0
SOCSTATUS_PENS_FL          0
REG_ADDRESS_PROVINCE       0
FACT_ADDRESS_PROVINCE      0
POSTAL_ADDRESS_PROVINCE    0
FL_PRESENCE_FL             0
OWN_AUTO                   0
FAMILY_INCOME              0
PERSONAL_INCOME            0
AGREEMENT_RK               0
TARGET                     0
LOAN_NUM_TOTAL             0
LOAN_NUM_CLOSED            0
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 0


In [254]:
final = df[[
    'AGREEMENT_RK', 'TARGET', 'AGE', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL',
    'GENDER', 'CHILD_TOTAL', 'DEPENDANTS', 'PERSONAL_INCOME', 'LOAN_NUM_TOTAL',
    'LOAN_NUM_CLOSED'
]].reset_index(drop=True)

main_info('Final dataset', final, 'AGREEMENT_RK', show_nans=True)


Final dataset: (15223, 11)
Number of NAs: 
AGREEMENT_RK         0
TARGET               0
AGE                  0
SOCSTATUS_WORK_FL    0
SOCSTATUS_PENS_FL    0
GENDER               0
CHILD_TOTAL          0
DEPENDANTS           0
PERSONAL_INCOME      0
LOAN_NUM_TOTAL       0
LOAN_NUM_CLOSED      0
dtype: int64

Number of duplicates: 0
Number of duplicates (only ID): 0


> **Комменатрий:** получили финальный датасет без дубликатов и значений, противоречащих друг другу, осталось только проверить, что все значения адекватны

In [255]:
final.describe()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
count,15223.00,15223.00,15223.00,15223.00,15223.00,15223.00,15223.00,15223.00,15223.00,15223.00,15223.00
mean,65401830.70,0.12,40.41,0.91,0.13,0.65,1.10,0.65,13853.84,1.39,0.75
std,4568181.14,0.32,11.60,0.29,0.34,0.48,1.00,0.81,9015.47,0.79,0.99
min,59910150.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,24.00,1.00,0.00
25%,61920915.00,0.00,30.00,1.00,0.00,0.00,0.00,0.00,8000.00,1.00,0.00
50%,64370996.00,0.00,39.00,1.00,0.00,1.00,1.00,0.00,12000.00,1.00,0.00
75%,67088017.50,0.00,50.00,1.00,0.00,1.00,2.00,1.00,17000.00,2.00,1.00
max,75292242.00,1.00,67.00,1.00,1.00,1.00,10.00,7.00,250000.00,11.00,11.00


> **Комменатрий:** все значения признаков выглядят адекватно, кроме некоторых значений PERSONAL_INCOME, так как среди них есть наблюдения, где доход человека исчисляется всего лишь в 24 рублях, необходимо взглянуть на это чуть подробней

In [256]:
final['PERSONAL_INCOME'].sort_values().head(10)

3203      24.00
14070   1950.00
2428    2000.00
7587    2000.00
11262   2100.00
12045   2300.00
8672    2800.00
9432    3000.00
6473    3000.00
7612    3000.00
Name: PERSONAL_INCOME, dtype: float64

In [257]:
final.drop(df[final['PERSONAL_INCOME'] == 24].index, inplace=True)
final.reset_index(drop=True, inplace=True)

> **Комменатрий:** убрали только наблюдение, где PERSONAL_INCOME==24, так как по остальным наблюдениям нельзяоднозначно сказать, что они неадекватны

In [258]:
final.describe()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
count,15222.00,15222.00,15222.00,15222.00,15222.00,15222.00,15222.00,15222.00,15222.00,15222.00,15222.00
mean,65402084.80,0.12,40.41,0.91,0.13,0.65,1.10,0.65,13854.74,1.39,0.75
std,4568223.62,0.32,11.60,0.29,0.34,0.48,1.00,0.81,9015.07,0.79,0.99
min,59910150.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,1950.00,1.00,0.00
25%,61920925.75,0.00,30.00,1.00,0.00,0.00,0.00,0.00,8000.00,1.00,0.00
50%,64371011.00,0.00,39.00,1.00,0.00,1.00,1.00,0.00,12000.00,1.00,0.00
75%,67088037.25,0.00,50.00,1.00,0.00,1.00,2.00,1.00,17000.00,2.00,1.00
max,75292242.00,1.00,67.00,1.00,1.00,1.00,10.00,7.00,250000.00,11.00,11.00


In [260]:
final.to_csv('datasets/final_dataset.csv', index=False)

## Задание 2

При помощи инструмента Streamlit проведите разведочный анализ данных. В него может входить:

* построение графиков распределений признаков
* построение матрицы корреляций
* построение графиков зависимостей целевой переменной и признаков
* вычисление числовых характеристик распределения числовых столбцов (среднее, min, max, медиана и так далее)
* любые другие ваши идеи приветствуются!

[Пример Streamlit-приложения](https://rateyourflight.streamlit.app) с разведочным анализом, прогнозом модели и оценкой ее результатов.

> **Комменатрий:** все в service/main.py